In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from config import API_KEY_INFO
service_key = API_KEY_INFO.get("seoul")

In [ ]:
class Transportation:
    """

    """
    def __init__(self, service_key=None):
        self.service_key = service_key
        self.endpoint = f"http://openapi.seoul.go.kr:8088/"
        self.metaDict = {
            "지하철승하차": {
                "url": f"{self.endpoint}{self.serviceKey}/xml/CardSubwayStatsNew/",
                "columns": ["USE_DT", "LINE_NUM", "SUB_STA_NM", "RIDE_PASGR_NUM", "ALIGHT_PASGR_NUM", "WORK_DT"]
            },
            "버스승하차": {
                "url": f"{self.endpoint}{self.serviceKey}/xml/CardBusStatisticsServiceNew/",
                "columns": ['USE_DT', 'BUS_ROUTE_ID', 'BUS_ROUTE_NO', 'BUS_ROUTE_NM', 'STND_BSST_ID', 'BSST_ARS_NO', 'BUS_STA_NM', 'RIDE_PASGR_NUM', 'ALIGHT_PASGR_NUM', 'WORK_DT']
            },
        }

    def get_data(self, category, translate=True, **kwargs):
        try:
            endpoint = self.metaDict[category]['url']
            columns = self.metaDict[category]['columns']
        except:
            _error_message = f"{category} 참조 오류"
            self.logger.error(_error_message)
            return _error_message

        try:
            params = ""
            for key, value in kwargs.items():
                params += f"/{value}"
        except:
            _error_message = f"{category} 파라미터 파싱 오류"
            self.logger.error(_error_message)
            return _error_message

        try:
            check_code = "INFO-000"
            startIdx, endIdx = 1, 1000
            items = []
            while check_code == "INFO-000":

                url = f"""{endpoint}{startIdx}/{endIdx}{params}"""

                # OpenAPI 호출
                result = requests.get(url, verify=False)
                xmlsoup = BeautifulSoup(result.text, "lxml-xml")
                header = xmlsoup.find("RESULT")
                result_code = header.find("CODE").text
                result_msg = header.find("MESSAGE").text
                rows = xmlsoup.findAll("row")
                items = items + rows

                check_code = result_code
                startIdx += 1000
                endIdx += 1000

        except:
            _error_message = f"HTTP 요청 혹은 파싱 오류"
            self.logger.error(_error_message)
            return _error_message

        # 데이터프레임 생성
        try:
            df = pd.DataFrame(columns=columns)
            for item in items:
                row = {}
                for col in columns:
                    try:
                        tag = item.find(col)
                        row[col] = tag.text.strip()
                    except:
                        row[col] = ""
                df_ = pd.DataFrame([row])[columns]
                df = pd.concat([df, df_], axis=0).reset_index(drop=True)

            df = self.ChangeCols(df)
            return df

        except:
            _error_message = f"전처리 오류"
            self.logger.error(_error_message)
            return _error_message

    def translate_columns(self, df):
        """
        영문 컬럼명을 국문 컬럼명으로 변경
        """

        self.colDict = {
            'USE_DT': '사용일자',

            'LINE_NUM': '호선명',
            'SUB_STA_NM': '역명',

            'BUS_ROUTE_ID': '노선ID',
            'BUS_ROUTE_NO': '노선번호',
            'BUS_ROUTE_NM': '노선명',
            'STND_BSST_ID': '표준버스정류장ID',
            'BSST_ARS_NO': '버스정류장ARS번호',
            'BUS_STA_NM': '역명',

            'RIDE_PASGR_NUM': '승차총승객수',
            'ALIGHT_PASGR_NUM': '하차총승객수',
            'WORK_DT': '등록일자'
        }

        df = df.rename(columns=self.colDict)
        return df


In [ ]:
api = Reb(service_key)